In [ ]:
def getmetadata(filename):
    import librosa
    import numpy as np

    y, sr = librosa.load(filename, sr=None)

    #fetching tempo
    onset_env = librosa.onset.onset_strength(y=y, sr=sr)

    # Extract features using librosa
    tempo, beats = librosa.beat.beat_track(y=y, sr=sr)
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    rmse = librosa.feature.rms(y=y)
    spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
    spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    zcr = librosa.feature.zero_crossing_rate(y)
    mfcc = librosa.feature.mfcc(y=y, sr=sr)

    metadata_dict = {'tempo':tempo,'chroma_stft':np.mean(chroma_stft),'rmse':np.mean(rmse), 'spectral_centroid':np.mean(spec_cent),'spectral_bandwidth':np.mean(spec_bw), 'rolloff':np.mean(rolloff), 'zero_crossing_rates':np.mean(zcr)}

    for i in range(1,21):
      metadata_dict.update({'mfcc'+str(i):np.mean(mfcc[i-1])})

    return list(metadata_dict.values())

In [ ]:
import pandas as pd
import numpy as np
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

In [ ]:

# Importing dataset
df = pd.read_csv('MGC_data1.csv')
df.head()

,class_name,tempo,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,blues,103.359375,0.380260,0.248262,2116.942959,1956.611056,4196.107960,0.127272,-26.929785,107.334008,...,14.336612,-13.821769,7.562789,-6.181372,0.330165,-6.829571,0.965922,-7.570825,2.918987,blues
1,blues,95.703125,0.306451,0.113475,1156.070496,1497.668176,2170.053545,0.058613,-233.860772,136.170239,...,-2.250578,3.959198,5.322555,0.812028,-1.107202,-4.556555,-2.436490,3.316913,-0.608485,blues
2,blues,151.999081,0.253487,0.151571,1331.073970,1973.643437,2900.174130,0.042967,-221.802549,110.843070,...,-13.037723,-12.652228,-1.821905,-7.260097,-6.660252,-14.682694,-11.719264,-11.025216,-13.387260,blues
3,blues,184.570312,0.269320,0.119072,1361.045467,1567.804596,2739.625101,0.069124,-207.208080,132.799175,...,-0.613248,0.384877,2.605128,-5.188924,-9.527455,-9.244394,-2.848274,-1.418707,-5.932607,blues
4,blues,161.499023,0.391059,0.137728,1811.076084,2052.332563,3927.809582,0.075480,-145.434568,102.829023,...,7.457218,-10.470444,-2.360483,-6.783624,2.671134,-4.760879,-0.949005,0.024832,-2.005315,blues


In [ ]:
# Preprocesing
print(df['class_name'].unique())
df['class_name'] = df['class_name'].astype('category')
df['class_label'] = df['class_name'].cat.codes

['blues' 'classical' 'country' 'disco' 'hiphop' 'jazz' 'metal' 'pop'
 'reggae' 'rock']


In [ ]:
lookup_genre_name = dict(zip(df.class_label.unique(), df.class_name.unique()))
lookup_genre_name

{0: 'blues',
 1: 'classical',
 2: 'country',
 3: 'disco',
 4: 'hiphop',
 5: 'jazz',
 6: 'metal',
 7: 'pop',
 8: 'reggae',
 9: 'rock'}

In [ ]:
#Displaying Feature Dataset
cols = list(df.columns)
cols.remove('label')
cols.remove('class_label')
cols.remove('class_name')
print(df[cols])

          tempo  chroma_stft      rmse  spectral_centroid  spectral_bandwidth  \
0    103.359375     0.380260  0.248262        2116.942959         1956.611056   
1     95.703125     0.306451  0.113475        1156.070496         1497.668176   
2    151.999081     0.253487  0.151571        1331.073970         1973.643437   
3    184.570312     0.269320  0.119072        1361.045467         1567.804596   
4    161.499023     0.391059  0.137728        1811.076084         2052.332563   
..          ...          ...       ...                ...                 ...   
995  123.046875     0.418812  0.166056        2312.030070         2060.825761   
996  117.453835     0.335774  0.093676        2553.856706         2219.090892   
997  103.359375     0.372834  0.208743        1345.850763         1841.748033   
998  112.347147     0.344472  0.180163        1675.475369         1653.422860   
999   75.999540     0.360098  0.042995        2209.357588         2175.214294   

         rolloff  zero_cros

In [ ]:
# Data splitting for Training and Testing
%matplotlib notebook
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
X = df.iloc[:,1:28]
y = df['class_label']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=3)

In [ ]:
# Min-Max Normalisation
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Model fitting using KNN
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 13)

knn.fit(X_train_scaled, y_train)

print("KNN Score: ",knn.score(X_test_scaled, y_test))

0.624

In [ ]:
a = getmetadata("Sample.wav")

# Prediction
d1 =np.array(a)
data1 = scaler.transform([d1])
genre_prediction = knn.predict(data1)
print("Gener of this song is :",lookup_genre_name[genre_prediction[0]])

Gener of this song is : classical


In [ ]:
a = getmetadata("Sample2.wav")

# Prediction
d1 =np.array(a)
data1 = scaler.transform([d1])
genre_prediction = knn.predict(data1)
print("Gener of this song is :",lookup_genre_name[genre_prediction[0]])

Gener of this song is : jazz


In [ ]:
a = getmetadata("Sample3.wav")

# Prediction
d1 =np.array(a)
data1 = scaler.transform([d1])
genre_prediction = knn.predict(data1)
print("Gener of this song is :",lookup_genre_name[genre_prediction[0]])

Gener of this song is : disco


In [ ]:
a = getmetadata("Sample4.wav")

# Prediction
d1 =np.array(a)
data1 = scaler.transform([d1])
genre_prediction = knn.predict(data1)
print("Gener of this song is :",lookup_genre_name[genre_prediction[0]])

Gener of this song is : pop
